In [1]:
import pandas as pd
import os
import tqdm
from collections import Counter
import numpy as np
from scipy.stats import multinomial
import numpy as np
import ast


In [58]:
# Load the data iteratively to avoid crashing the kernel
df = pd.read_csv("/Users/reem/Mov/nextclade_results/llr_top_1000.tsv",sep="\t")
df.head()


,Unnamed: 0,index,seqName,clade,clade_display,clade_who,clade_nextstrain,partiallyAliased,Nextclade_pango,qc.overallScore,...,pcrPrimerChanges,failedCdses,warnings,errors,subs,Counts,LLR,Country,context,spectrum
0,0,2,hCoV-19/USA/MO-WRAIR-COX5040NPS/2020|2020-08-1...,20B,20B,NaN,20B,B.1.1,B.1.1,197.666876,...,NaN,NaN,NaN,NaN,"G>A,C>T,C>T,C>T,C>T,C>T,C>T,G>A,C>T,A>C,A>T,G>...","Counter({'C>T': 8, 'G>A': 3, 'A>G': 3, 'A>C': ...",-0.378720,USA,"CGT,ACT,TCT,ACC,CCT,GCT,TCA,AGG,TCA,GAG,CAA,TG...","C[G>A]T,A[C>T]T,T[C>T]T,A[C>T]C,C[C>T]T,G[C>T]..."
1,1,1,hCoV-19/Belgium/UGent-14493/2021|2021-12-22|20...,21K,21K (BA.1),Omicron,21K,BA.1.14,BA.1.14,2.790137,...,NaN,NaN,NaN,NaN,NaN,Counter(),0.000000,Belgium,NaN,NaN
2,2,7,hCoV-19/France/IDF-HMN-21052200412/2021|2021-0...,21A,21A (Delta),Delta,21A,B.1.617.2,B.1.617.2,0.000000,...,NaN,NaN,NaN,NaN,NaN,Counter(),0.000000,France,NaN,NaN
3,3,8,hCoV-19/England/LSPA-37EF052/2022|2022-02-21|2...,21L,21L (BA.2),Omicron,21L,BA.2,BA.2,0.000000,...,NaN,NaN,NaN,NaN,G>T,Counter({'G>T': 1}),-1.509730,England,AGT,A[G>T]T
4,4,5,hCoV-19/Germany/BE-RKI-I-595719/2022|2022-02-2...,21L,21L (BA.2),Omicron,21L,BA.2,BA.2,0.000000,...,NaN,NaN,NaN,NaN,"G>T,A>G","Counter({'G>T': 1, 'A>G': 1})",-1.720178,Germany,"TGG,AAT","T[G>T]G,A[A>G]T"


In [50]:
df_low = df[df['LLR']<6]
df_low.head()

,Unnamed: 0,index,seqName,clade,clade_display,clade_who,clade_nextstrain,partiallyAliased,Nextclade_pango,qc.overallScore,...,pcrPrimerChanges,failedCdses,warnings,errors,subs,Counts,LLR,Country,context,spectrum
0,0,2,hCoV-19/USA/MO-WRAIR-COX5040NPS/2020|2020-08-1...,20B,20B,NaN,20B,B.1.1,B.1.1,197.666876,...,NaN,NaN,NaN,NaN,"G>A,C>T,C>T,C>T,C>T,C>T,C>T,G>A,C>T,A>C,A>T,G>...","Counter({'C>T': 8, 'G>A': 3, 'A>G': 3, 'A>C': ...",-0.378720,USA,"CGT,ACT,TCT,ACC,CCT,GCT,TCA,AGG,TCA,GAG,CAA,TG...","C[G>A]T,A[C>T]T,T[C>T]T,A[C>T]C,C[C>T]T,G[C>T]..."
1,1,1,hCoV-19/Belgium/UGent-14493/2021|2021-12-22|20...,21K,21K (BA.1),Omicron,21K,BA.1.14,BA.1.14,2.790137,...,NaN,NaN,NaN,NaN,NaN,Counter(),0.000000,Belgium,NaN,NaN
2,2,7,hCoV-19/France/IDF-HMN-21052200412/2021|2021-0...,21A,21A (Delta),Delta,21A,B.1.617.2,B.1.617.2,0.000000,...,NaN,NaN,NaN,NaN,NaN,Counter(),0.000000,France,NaN,NaN
3,3,8,hCoV-19/England/LSPA-37EF052/2022|2022-02-21|2...,21L,21L (BA.2),Omicron,21L,BA.2,BA.2,0.000000,...,NaN,NaN,NaN,NaN,G>T,Counter({'G>T': 1}),-1.509730,England,AGT,A[G>T]T
4,4,5,hCoV-19/Germany/BE-RKI-I-595719/2022|2022-02-2...,21L,21L (BA.2),Omicron,21L,BA.2,BA.2,0.000000,...,NaN,NaN,NaN,NaN,"G>T,A>G","Counter({'G>T': 1, 'A>G': 1})",-1.720178,Germany,"TGG,AAT","T[G>T]G,A[A>G]T"


In [59]:
#Get Counts per substitution context
def count_CtoT(spectrum):
    counts = Counter()
    if not isinstance(spectrum, str):
        return {}
    muts = spectrum.split(",")
    for mut in muts:
        if mut[2:5] == 'C>T':
            counts[mut]+=1
    return counts

def count_TtoC(spectrum):
    counts = Counter()
    if not isinstance(spectrum, str):
        return {}
    muts = spectrum.split(",")
    for mut in muts:
        if mut[2:5] == 'T>C':
            counts[mut]+=1
    return counts

def count_AtoG(spectrum):
    counts = Counter()
    if not isinstance(spectrum, str):
        return {}
    muts = spectrum.split(",")
    for mut in muts:
        if mut[2:5] == 'A>G':
            counts[mut]+=1
    return counts

df["T>C_counts"] = df["spectrum"].apply(count_TtoC)
df["C>T_counts"] = df["spectrum"].apply(count_CtoT)
df["A>G_counts"] = df["spectrum"].apply(count_AtoG)
print(df["T>C_counts"].head(6))
print(df["C>T_counts"].head())
print(df["A>G_counts"].head())


0                                            {}
1                                            {}
2                                            {}
3                                            {}
4                                            {}
5    {'G[T>C]T': 1, 'A[T>C]G': 1, 'A[T>C]A': 1}
Name: T>C_counts, dtype: object
0    {'A[C>T]T': 1, 'T[C>T]T': 1, 'A[C>T]C': 1, 'C[...
1                                                   {}
2                                                   {}
3                                                   {}
4                                                   {}
Name: C>T_counts, dtype: object
0    {'A[A>G]A': 3}
1                {}
2                {}
3                {}
4    {'A[A>G]T': 1}
Name: A>G_counts, dtype: object


In [60]:
def get_proportion(df):
    dict = {}
    total = sum(df.values())
    for key, value in df.items():
        dict[key] = value/total
       
    return dict

df["T>Cproportions"] = df.apply(lambda row: get_proportion(row["T>C_counts"]), axis=1)
df["C>Tproportions"] = df.apply(lambda row: get_proportion(row["C>T_counts"]), axis=1)
df["A>Gproportions"] = df.apply(lambda row: get_proportion(row["A>G_counts"]), axis=1)

print(df["T>Cproportions"].head(10))
print(df["C>Tproportions"].head(10))
print(df["A>Gproportions"].head(10))



0                                                   {}
1                                                   {}
2                                                   {}
3                                                   {}
4                                                   {}
5    {'G[T>C]T': 0.3333333333333333, 'A[T>C]G': 0.3...
6                                                   {}
7                                                   {}
8                                     {'C[T>C]C': 1.0}
9                                                   {}
Name: T>Cproportions, dtype: object
0    {'A[C>T]T': 0.125, 'T[C>T]T': 0.125, 'A[C>T]C'...
1                                                   {}
2                                                   {}
3                                                   {}
4                                                   {}
5                     {'T[C>T]T': 0.5, 'G[C>T]T': 0.5}
6                                                   {}
7    {'T[C>T]A': 0.5, 'T[C>T]

In [75]:
df_high = df[df['LLR']>2]
df_high.head()

,Unnamed: 0,index,seqName,clade,clade_display,clade_who,clade_nextstrain,partiallyAliased,Nextclade_pango,qc.overallScore,...,LLR,Country,context,spectrum,T>C_counts,C>T_counts,A>G_counts,T>Cproportions,C>Tproportions,A>Gproportions
26,26,23,hCoV-19/England/QEUH-9E4607/2020|2020-09-23|20...,20A,20A,NaN,20A,B.1.416.1,B.1.416.1,0.173611,...,2.249892,England,"ACA,ACA,ACT,GGT,ACG","A[C>T]A,A[C>T]A,A[C>T]T,G[G>A]T,A[C>T]G",{},"{'A[C>T]A': 2, 'A[C>T]T': 1, 'A[C>T]G': 1}",{},{},"{'A[C>T]A': 0.5, 'A[C>T]T': 0.25, 'A[C>T]G': 0...",{}
89,89,85,hCoV-19/Netherlands/GL-RIVM-78670/2021|2021-12...,21J,21J (Delta),Delta,21J,B.1.617.2.46.6,AY.46.6,14.062500,...,3.347984,Netherlands,"ACG,ACA,TGT,ACG,CCT,CCT,TCG,AAT,ACG","A[C>T]G,A[C>T]A,T[G>A]T,A[C>T]G,C[C>T]T,C[C>T]...",{},"{'A[C>T]G': 3, 'A[C>T]A': 1, 'C[C>T]T': 2, 'T[...",{'A[A>G]T': 1},{},"{'A[C>T]G': 0.42857142857142855, 'A[C>T]A': 0....",{'A[A>G]T': 1.0}
242,242,244,hCoV-19/USA/VT-CDCBI-CRSP_FQVRTUGRSIY24EZD/202...,21J,21J (Delta),Delta,21J,B.1.617.2.44,AY.44,4.340278,...,2.048701,USA,"TCA,TCA,GCA,ACA,TCT,ACA,ACG,TGG,TGT","T[C>T]A,T[C>T]A,G[C>T]A,A[C>T]A,T[C>T]T,A[C>T]...",{},"{'T[C>T]A': 2, 'G[C>T]A': 1, 'A[C>T]A': 2, 'T[...",{},{},"{'T[C>T]A': 0.2857142857142857, 'G[C>T]A': 0.1...",{}
298,298,297,hCoV-19/France/PDL-IPP16500/2023|2023-07-23|20...,23A,23A (XBB.1.5),NaN,23A,XBB.1.5.59,XBB.1.5.59,4.340278,...,2.249892,France,"GCA,GGT,ACA,TCG,ACA","G[C>T]A,G[G>A]T,A[C>T]A,T[C>T]G,A[C>T]A",{},"{'G[C>T]A': 1, 'A[C>T]A': 2, 'T[C>T]G': 1}",{},{},"{'G[C>T]A': 0.25, 'A[C>T]A': 0.5, 'T[C>T]G': 0...",{}
364,364,361,hCoV-19/Canada/BC-BCCDC-694463/2023|2023-12-22...,23F,23F (EG.5.1),NaN,23F,XBB.1.9.2.5.1.6.1,HV.1,0.000000,...,2.039445,Canada,"ACA,CGT,GCT,TCT,ACA,CAT","A[C>T]A,C[G>A]T,G[C>T]T,T[C>T]T,A[C>T]A,C[A>G]T",{},"{'A[C>T]A': 2, 'G[C>T]T': 1, 'T[C>T]T': 1}",{'C[A>G]T': 1},{},"{'A[C>T]A': 0.5, 'G[C>T]T': 0.25, 'T[C>T]T': 0...",{'C[A>G]T': 1.0}


In [76]:
ct_pivot = pd.json_normalize(df_high['T>Cproportions'])
ct_pivot.index = df_high.index 
ct_pivot = ct_pivot.fillna(0)
df_Mov = pd.concat([df_high[["seqName"]],ct_pivot,df_high[["LLR"]]], axis=1)
df_Mov.head(20)

,seqName,A[T>C]G,A[T>C]T,C[T>C]C,A[T>C]A,A[T>C]C,LLR
26,hCoV-19/England/QEUH-9E4607/2020|2020-09-23|20...,0.0,0.0,0.0,0.0,0.0,2.249892
89,hCoV-19/Netherlands/GL-RIVM-78670/2021|2021-12...,0.0,0.0,0.0,0.0,0.0,3.347984
242,hCoV-19/USA/VT-CDCBI-CRSP_FQVRTUGRSIY24EZD/202...,0.0,0.0,0.0,0.0,0.0,2.048701
298,hCoV-19/France/PDL-IPP16500/2023|2023-07-23|20...,0.0,0.0,0.0,0.0,0.0,2.249892
364,hCoV-19/Canada/BC-BCCDC-694463/2023|2023-12-22...,0.0,0.0,0.0,0.0,0.0,2.039445
446,hCoV-19/USA/ND-CDC-IBX888198659281/2022|2022-0...,0.0,0.0,0.0,0.0,0.0,2.180899
447,hCoV-19/USA/OH-CDC-LC0054872/2021|2021-04-30|2...,1.0,0.0,0.0,0.0,0.0,2.313910
582,hCoV-19/England/QEUH-2A524B8/2021|2021-11-12|2...,0.0,1.0,0.0,0.0,0.0,2.617892
638,hCoV-19/USA/OH-ODH-SC2044455/2022|2022-01-31|2...,0.0,0.0,0.0,0.0,0.0,2.249892
661,hCoV-19/USA/CA-CDPH-3000276276/2021|2021-11-24...,0.0,0.0,0.0,0.0,0.0,2.318885


In [77]:
context_cols = [col for col in df_Mov.columns if col not in ["seqName", "LLR"]]
mean_row = df_Mov[context_cols].fillna(0).mean()
mean_row['seqName'] = 'Mean'
#print(mean_row)

df_Mov = pd.concat([df_Mov, pd.DataFrame([mean_row])], ignore_index=True)
print(df_Mov.tail())

                                              seqName   A[T>C]G  A[T>C]T  \
11  hCoV-19/India/MZ-INSACOG-561253365/2021|2021-1...  0.000000      0.0   
12  hCoV-19/Germany/NI-RKI-I-1115367/2023|2023-02-...  0.000000      0.0   
13  hCoV-19/USA/FL-Miami-1014/2021|2021-03|2021-09-28  0.000000      0.0   
14  hCoV-19/USA/CA-CDC-FG-161790/2021|2021-11-01|2...  0.000000      0.0   
15                                               Mean  0.066667      0.1   

     C[T>C]C   A[T>C]A   A[T>C]C       LLR  
11  0.000000  1.000000  0.000000  2.040258  
12  0.000000  1.000000  0.000000  2.593351  
13  0.000000  0.000000  1.000000  2.088178  
14  0.000000  0.000000  0.000000  2.686072  
15  0.033333  0.133333  0.066667       NaN  


In [136]:
df_low = df[df["LLR"]<6]
df_low.head()


,Unnamed: 0,index,seqName,clade,clade_display,clade_who,clade_nextstrain,partiallyAliased,Nextclade_pango,qc.overallScore,...,Country,context,spectrum,T>C_counts,C>T_counts,A>G_counts,T>Cproportions,C>Tproportions,A>Gproportions,T>C_llr
0,0,2,hCoV-19/USA/MO-WRAIR-COX5040NPS/2020|2020-08-1...,20B,20B,NaN,20B,B.1.1,B.1.1,197.666876,...,USA,"CGT,ACT,TCT,ACC,CCT,GCT,TCA,AGG,TCA,GAG,CAA,TG...","C[G>A]T,A[C>T]T,T[C>T]T,A[C>T]C,C[C>T]T,G[C>T]...",{},"{'A[C>T]T': 1, 'T[C>T]T': 1, 'A[C>T]C': 1, 'C[...",{'A[A>G]A': 3},{},"{'A[C>T]T': 0.125, 'T[C>T]T': 0.125, 'A[C>T]C'...",{'A[A>G]A': 1.0},NaN
1,1,1,hCoV-19/Belgium/UGent-14493/2021|2021-12-22|20...,21K,21K (BA.1),Omicron,21K,BA.1.14,BA.1.14,2.790137,...,Belgium,NaN,NaN,{},{},{},{},{},{},NaN
2,2,7,hCoV-19/France/IDF-HMN-21052200412/2021|2021-0...,21A,21A (Delta),Delta,21A,B.1.617.2,B.1.617.2,0.000000,...,France,NaN,NaN,{},{},{},{},{},{},NaN
3,3,8,hCoV-19/England/LSPA-37EF052/2022|2022-02-21|2...,21L,21L (BA.2),Omicron,21L,BA.2,BA.2,0.000000,...,England,AGT,A[G>T]T,{},{},{},{},{},{},NaN
4,4,5,hCoV-19/Germany/BE-RKI-I-595719/2022|2022-02-2...,21L,21L (BA.2),Omicron,21L,BA.2,BA.2,0.000000,...,Germany,"TGG,AAT","T[G>T]G,A[A>G]T",{},{},{'A[A>G]T': 1},{},{},{'A[A>G]T': 1.0},NaN


In [ ]:
ct_pivot = pd.json_normalize(df_low['T>Cproportions'])
ct_pivot.index = df_low.index 
ct_pivot = ct_pivot.fillna(0)
print(ct_pivot.shape)
print(ct_pivot.columns[:20].tolist())
print(ct_pivot.sum(axis=1).head(10).tolist())
print(ct_pivot.sum(axis=1).mean())
# df_Normal = pd.concat([df_low[["seqName"]],ct_pivot,df_low[["LLR"]]], axis=1)
# df_Normal.head(20)
# context_cols = [col for col in df_Mov.columns if col not in ["seqName", "LLR"]]
# mean_row = df_Normal[context_cols].fillna(0).mean()
# mean_row['seqName'] = 'Mean'
# #print(mean_row)

# df_Normal = pd.concat([df_Normal, pd.DataFrame([mean_row])], ignore_index=True)
# print(df_Normal)

ct shape: (999, 16)
First columns: ['G[T>C]T', 'A[T>C]G', 'A[T>C]A', 'C[T>C]C', 'T[T>C]A', 'A[T>C]C', 'C[T>C]G', 'T[T>C]C', 'G[T>C]G', 'T[T>C]G', 'T[T>C]T', 'C[T>C]T', 'G[T>C]C', 'C[T>C]A', 'A[T>C]T', 'G[T>C]A']
Row sums (first 10): [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]
Mean of row sums (should be ~1): 0.36736736736736736


In [119]:
def generate_all_possible_contexts(sub):
    bases = ["A","C","G","T"]
    possible_contexts = []
    for a in bases:
        for b in bases:
            possible_contexts.append(f"{a}[{sub}]{b}")
    return possible_contexts
print(generate_all_possible_contexts("G>A"))

['A[G>A]A', 'A[G>A]C', 'A[G>A]G', 'A[G>A]T', 'C[G>A]A', 'C[G>A]C', 'C[G>A]G', 'C[G>A]T', 'G[G>A]A', 'G[G>A]C', 'G[G>A]G', 'G[G>A]T', 'T[G>A]A', 'T[G>A]C', 'T[G>A]G', 'T[G>A]T']


In [155]:
def get_mean_context_probs(df, llr_condition, prop_col, llr_col='LLR'):
    """
    Filters dataframe into llr>6 and llr<6 and computes mean 
    proportions per substitution per context

    Returns a flattened dataframe with mutational contexts,seqname,LLR
    and computed means as the last row

    """

    df_filtered = df[llr_condition(df[llr_col])].copy()
    
    ct_pivot = pd.json_normalize(df_filtered[prop_col]).fillna(0)
    ct_pivot = ct_pivot.loc[(ct_pivot != 0).any(axis=1)]
    #ct_pivot = ct_pivot.div(ct_pivot.sum(axis=1), axis=0)
    # ct_pivot.index = df_filtered.index

    return ct_pivot.mean()

#print(get_mean_context_probs(df, llr_condition=lambda x: x > 1, prop_col="T>Cproportions", llr_col='LLR'))
def build_prob_table(df,prop_col,llr_high,llr_low,llr_col='LLR'):
    """
    Builds a table of mean context probabilities (Molnupiravir vs Normal)
    for a specific substitution type.
    """
    llr_high=2
    llr_low=6
   
    Mov_probs = get_mean_context_probs(df.loc[(df!=0).any(axis=1)], lambda x: x > 1, prop_col='C>Tproportions')
    Normal_probs  = get_mean_context_probs(df.loc[(df!=0).any(axis=1)], lambda x: x < 6, prop_col='C>Tproportions')
    all_contexts = generate_all_possible_contexts("C>T")
    
    # Match means to all possible contexts
    Mov_probs = Mov_probs.reindex(all_contexts, fill_value=0)
    Normal_probs = Normal_probs.reindex(all_contexts, fill_value=0)

    print(Mov_probs)
    print(Normal_probs)

    df_prob = pd.DataFrame({'Mutational_Context':all_contexts,
                           'Molnupiravir': Mov_probs.values,
                            'Normal': Normal_probs.values
                            }) 

    
    return df_prob


df_prob = build_prob_table(df,prop_col='C>Tproportions',llr_high=1, llr_low=6, llr_col='LLR')

print(df_prob)



A[C>T]A    0.156162
A[C>T]C    0.044549
A[C>T]G    0.071304
A[C>T]T    0.125028
C[C>T]A    0.052203
C[C>T]C    0.016258
C[C>T]G    0.015263
C[C>T]T    0.040095
G[C>T]A    0.047806
G[C>T]C    0.014076
G[C>T]G    0.008088
G[C>T]T    0.065336
T[C>T]A    0.102961
T[C>T]C    0.033536
T[C>T]G    0.074292
T[C>T]T    0.133042
dtype: float64
A[C>T]A    0.165108
A[C>T]C    0.053990
A[C>T]G    0.048534
A[C>T]T    0.142269
C[C>T]A    0.069174
C[C>T]C    0.017848
C[C>T]G    0.011171
C[C>T]T    0.038246
G[C>T]A    0.037279
G[C>T]C    0.018556
G[C>T]G    0.017143
G[C>T]T    0.076089
T[C>T]A    0.095760
T[C>T]C    0.025858
T[C>T]G    0.071524
T[C>T]T    0.111450
dtype: float64
   Mutational_Context  Molnupiravir    Normal
0             A[C>T]A      0.156162  0.165108
1             A[C>T]C      0.044549  0.053990
2             A[C>T]G      0.071304  0.048534
3             A[C>T]T      0.125028  0.142269
4             C[C>T]A      0.052203  0.069174
5             C[C>T]C      0.016258  0.017848
6       

In [156]:
print(df_prob)
print(df_prob["Molnupiravir"].sum())
print(df_prob["Normal"].sum())

   Mutational_Context  Molnupiravir    Normal
0             A[C>T]A      0.156162  0.165108
1             A[C>T]C      0.044549  0.053990
2             A[C>T]G      0.071304  0.048534
3             A[C>T]T      0.125028  0.142269
4             C[C>T]A      0.052203  0.069174
5             C[C>T]C      0.016258  0.017848
6             C[C>T]G      0.015263  0.011171
7             C[C>T]T      0.040095  0.038246
8             G[C>T]A      0.047806  0.037279
9             G[C>T]C      0.014076  0.018556
10            G[C>T]G      0.008088  0.017143
11            G[C>T]T      0.065336  0.076089
12            T[C>T]A      0.102961  0.095760
13            T[C>T]C      0.033536  0.025858
14            T[C>T]G      0.074292  0.071524
15            T[C>T]T      0.133042  0.111450
1.0
0.9999999999999999


In [157]:
#Calculate LLRs using prob_table:

def calculate_llr(count_dict, pM, pN, contexts):
    counts = np.array([count_dict.get(ctx, 0) for ctx in contexts])
    n = counts.sum()
    if n == 0:
        return np.nan
    llM = multinomial.logpmf(counts, n=n, p=pM)
    llN = multinomial.logpmf(counts, n=n, p=pN)
    return float(llM - llN)

contexts = df_prob["Mutational_Context"].values.tolist()
pM = df_prob["Molnupiravir"].values.tolist()
pN = df_prob["Normal"].values.tolist()
df[f"C>T_llr"] = df["C>T_counts"].apply(lambda x: calculate_llr(x, pM, pN, contexts))

print(df[f"C>T_llr"])

# === Save results ===
#df[["seqName", "LLR_C>T"]].to_csv("LLR_CtoT.tsv", sep="\t", index=False)



0      0.280278
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
994         NaN
995    0.121376
996    0.072512
997    0.130792
998   -0.152351
Name: C>T_llr, Length: 999, dtype: float64
